# Practice Exercises for Single Table SQL

Given a dataset and a set of questions, form the queries to answer the questions. All questions in this particular exercise only require a query to a single table. 

⚠️ *Note:* Please display minimal number of attributes (columns) needed to make sense of your answer. Not too many, not too few.


## Setup
The SQLite database for these exercises has been setup for you. For the curious, you can look at this file for the DDL commands to setup the tables and insert data.  `db_illum_ddl_5_3.sql`

Here is the schema for the database. The arrows show foreign keys, which is just fyi for now. 

![worker_project schema](worker_project_schema.png) 

In [55]:
# Make sure to always run this cell when you open this notebook
# Start the Jupyter SQL engine, connecting to a SQLite database 
%reload_ext sql 
%sql sqlite:///worker_project.db

0. In the command line, inspect the sqlite database (worker_project.db) to see what the tables and columns actually are. Any differences you observe? (Also, copy past your output in the cell below)

In [ ]:
sqlite> SELECT * FROM worker;
101|Smith|Tom|Accounting|01-Feb-1970|06-Jun-1993 |50000
103|Jones|Mary|Accounting|15-Jun-1975|20-Sep-2005|48000
105|Burns|Jane|Accounting|21-Sep-1980|12-Jun-2015|39000
110|Burns|Michael|Research|05-Apr-1977|10-Sep-2020|70000
115|Chin|Amanda|Research|22-Sep-1980|19-Jun-2022|60000
sqlite> .table
Assign   Dept     Project  Worker 
sqlite> .schema
CREATE TABLE Worker (
empId NUMBER(6) PRIMARY KEY, 
lastName VARCHAR2(20) NOT NULL,
firstName VARCHAR2(15) NOT NULL, 
deptName VARCHAR2(15), 
birthDate DATE, 
hireDate DATE, 
salary NUMBER(8,2));
CREATE TABLE Dept(
deptName VARCHAR2(15),
mgrId NUMBER(6),
CONSTRAINT Dept_deptName_pk PRIMARY KEY (deptName),
CONSTRAINT Dept_mgrId_fk FOREIGN KEY (mgrId) REFERENCES Worker(empId) ON DELETE SET NULL);
CREATE TABLE Project (
projNo NUMBER(6), 
projName VARCHAR2(20), 
projMgrId NUMBER(6),
budget NUMBER (8,2), 
startDate DATE, 
expectedDurationWeeks NUMBER(4),
CONSTRAINT Project_projNo_pk PRIMARY KEY (projNo),
CONSTRAINT Project_projMgrId_fk FOREIGN KEY(projMgrId) REFERENCES WORKER(empId) ON DELETE SET NULL);
CREATE TABLE Assign (
projNo NUMBER(6), 
empId NUMBER(6), 
hoursAssigned NUMBER(3), 
rating NUMBER(1),
CONSTRAINT Assign_projNo_empId_pk PRIMARY KEY (projNo, empId),
CONSTRAINT Assign_projNo_fk FOREIGN KEY(projNo) REFERENCES Project(projNo) ON DELETE CASCADE,
CONSTRAINT Assign_empId_fk FOREIGN KEY(empId) REFERENCES Worker(empId) ON DELETE CASCADE);
sqlite> SELECT * Dept;
Parse error: near "Dept": syntax error
  SELECT * Dept;
           ^--- error here
sqlite> SELECT * FROM Dept;
Accounting|101
Research|110
sqlite> SELECT * FROM Worker;
101|Smith|Tom|Accounting|01-Feb-1970|06-Jun-1993 |50000
103|Jones|Mary|Accounting|15-Jun-1975|20-Sep-2005|48000
105|Burns|Jane|Accounting|21-Sep-1980|12-Jun-2015|39000
110|Burns|Michael|Research|05-Apr-1977|10-Sep-2020|70000
115|Chin|Amanda|Research|22-Sep-1980|19-Jun-2022|60000
sqlite> SELECT * FROM Project;
1001|Jupiter|101|300000|01-Feb-2021|50
1005|Saturn|101|400000|01-Jun-2022|35
1019|Mercury|110|350000|15-Feb-2022|40
1025|Neptune|110|600000|01-Feb-2021|45
1030|Pluto|110|380000|15-Sep-2023|50
sqlite> SELECT * FROM Assign;
1001|101|30|
1001|103|20|5
1005|103|20|
1001|105|30|
1001|115|20|4
1019|110|20|5
1019|115|10|4
1025|110|10|
1030|110|10|

1. Get the names of all workers in the Accounting department.

In [7]:
%%sql
SELECT firstName|| " " || lastName AS FullName FROM Worker WHERE deptName ='Accounting';



 * sqlite:///worker_project.db
Done.


FullName
Tom Smith
Mary Jones
Jane Burns


2. Get the name of the employee in the Research department who has the lowest salary.

In [2]:
%%sql
-- SELECT firstName || " " || lastName AS FullName FROM Worker WHERE deptName ='Research' AND salary =MIN;
SELECT firstName, lastName FROM Worker WHERE deptName = 'Research' ORDER BY salary ASC LIMIT 1;


 * sqlite:///worker_project.db
Done.


firstName,lastName
Amanda,Chin


3. Get the details of the project with the highest budget.

In [9]:
%%sql
SELECT * FROM Project WHERE budget = (SELECT MAX(budget) FROM Project);


 * sqlite:///worker_project.db
Done.


projNo,projName,projMgrId,budget,startDate,expectedDurationWeeks
1025,Neptune,110,600000,01-Feb-2021,45


## The following questions may require more than one query to answer. 
Show your work! Add as many SQL cells as needed for each question. Use comments to indicate how the output of each query is used to arrive at your answer.  

4. List out alphabetically the names of all workers working on Project 1001.

In [13]:
%%sql
SELECT firstName, lastName from Worker, Assign ON Worker.empId = Assign.empId
WHERE
projNo = 1001 
ORDER BY firstName, lastName ASC








 * sqlite:///worker_project.db
Done.


firstName,lastName
Amanda,Chin
Jane,Burns
Mary,Jones
Tom,Smith


5. Get an alphabetical list of names and corresponding ratings for all workers on any project managed by Michael Burns.

In [51]:
%%sql

SELECT firstName, lastName, rating FROM (SELECT empId, rating FROM Assign WHERE projNo = (SELECT projNo FROM Project WHERE projMgrId = (SELECT empId FROM Worker WHERE firstName ="Michael" AND lastName ="Burns")
)) AS t
LEFT JOIN Worker ON Worker.empId = t.empId



 * sqlite:///worker_project.db
Done.


firstName,lastName,rating
Michael,Burns,5
Amanda,Chin,4


6. For all projects starting after Feb 1, 2022, find the project number and names of all workers assigned to them. 

In [64]:
%%sql
SELECT projNo FROM project WHERE startDate > "01-Feb-2022"

 * sqlite:///worker_project.db
Done.


projNo
1005
1019
1030


In [75]:
%%sql
SELECT firstName, lastName FROM Worker WHERE empId IN 
(SELECT empId FROM Assign WHERE projNo 
in (SELECT projNo FROM project WHERE startDate > "01-Feb-2022"))



 * sqlite:///worker_project.db
Done.


firstName,lastName
Mary,Jones
Michael,Burns
Amanda,Chin


7. Find the names of employees who are not assigned as managers any project.

8. Find details of any project with the string "urn" in the project name. 

9. List out the projects from most to least efficient. (hint: the project with the lowest budget is not necessarily the one that uses time and human resources effectively.)

10. Explain your reasoning for your solution to #9 (use a Markdown cell), including who should get the award for Most Efficient Project Manager.

11. Bonus!!  A Query that you can't easily answer with just a set of queries: Get a list of project numbers with name and start dates, of all the projects that have the same start date as another. 
*you don't have to answer this, but think about it!*


## Submission:

Commit and push your changes to this document by the deadline, and share a link to the repository in the Moodle assignment.  